In [256]:
import gurobipy as gp

from gurobipy import GRB
import numpy as np

# Farm Planning, problem 1
First, we create a LP problem for the first period--Janurary.

there's six decision variables: VEG1,VEG2,OIL1,OIL2,OIL3 and FINPROD. Our single period objective function is to minimize their costs and maximize their profit. 
$$P^t_{x_1} : \text{represents the price of X_1 at time t}$$


Our objective function is to maximize profit-- in other words, find the optimal comibination of all blended materials subtracted from the total profit of the finished product.

For a single time period t, the objective function is:
$$\text{max} \{ -P^t_{x_1}X_1 - P^t_{x_2} X_2 - P^t_{x_3}X_3 - P^t_{x_4}X_4 - P^t_{x_5}X_5 + 150 Y \} $$, where $Y$ is the amount of finished product.

We have to ensure that the hardness of the blend is kept within acceptable limits. 

$$ 8.8 X_1 + 6.1X_2 + 2.0X_3 + 4.2X_4 + 5.0X_5 - 3Y \leq 0$$ (lower bound hardness constraint)

$$ 8.8 X_1 + 6.1X_2 + 2.0X_3 + 4.2X_4 + 5.0X_5 - 6Y \leq 0$$ (upper bound hardness constraint)


We also need to ensure that there's continuity between the finished and final product, by adding a contuinity constraint:
$$X_1 + X_2 + X_3 + X_4 + X_5 - Y = 0$$

We also have the production constraints!

$$ X_1 + X_2 - 200 \leq 0 $$  
and
$$ X_3 + X_4 + X_5 -250\leq 0$$
We can't produce negative amounts of ingredients!

$$ X_i \geq 0 : i \in \{1,2,3,4,5\} $$

In [265]:
# Loading the data in first
prices = gp.tupledict({
	('January', 'VEG1'): 110,
	('January', 'VEG2'): 120,
	('January', 'OIL1'): 130,
	('January', 'OIL2'): 110,
	('January', 'OIL3'): 115,
	('February', 'VEG1'): 130,
	('February', 'VEG2'): 130,
	('February', 'OIL1'): 110,
	('February', 'OIL2'): 90,
	('February', 'OIL3'): 115,
	('March', 'VEG1'): 110,
	('March', 'VEG2'): 140,
	('March', 'OIL1'): 130,
	('March', 'OIL2'): 100,
	('March', 'OIL3'): 95,
	('April', 'VEG1'): 120,
	('April', 'VEG2'): 110,
	('April', 'OIL1'): 120,
	('April', 'OIL2'): 120,
	('April', 'OIL3'): 125,
	('May', 'VEG1'): 100,
	('May', 'VEG2'): 120,
	('May', 'OIL1'): 150,
	('May', 'OIL2'): 110,
	('May', 'OIL3'): 105,
	('June', 'VEG1'): 90,
	('June', 'VEG2'): 100,
	('June', 'OIL1'): 140,
	('June', 'OIL2'): 80,
	('June', 'OIL3'): 135
})

hardness_vals = gp.tupledict([('VEG1',8.8),('VEG2',6.1),('OIL1',2.0),('OIL2',4.2),('OIL3',5.0)])

decision_names = ['VEG1','VEG2','OIL1','OIL2','OIL3','FINPROD']


['VEG1', 'VEG2', 'OIL1', 'OIL2', 'OIL3']

In [266]:
#building model for janurary.
current_month = 'January'
try:
    m = gp.Model()
    #add decision vars
    d_vars = m.addVars(decision_names,vtype=GRB.CONTINUOUS,name='used')
    #add objective function
    m.update()
    m.setObjective( (-1)*gp.quicksum(
        prices[(current_month,prod)] * d_vars[prod] 
                                     for prod in decision_names[0:-1] ) + 150 *d_vars['FINPROD'],
                   GRB.MAXIMIZE )
    #add hardness constraints
    hard_up = m.addConstr( gp.quicksum(hardness_vals[prod]*d_vars[prod] for prod in decision_names[0:-1])+ -6*d_vars['FINPROD'] >= 0)
    hard_low = m.addConstr( gp.quicksum(hardness_vals[prod]*d_vars[prod] for prod in decision_names[0:-1])+ -3*d_vars['FINPROD'] >= 0)
    
    #add continuity constraints
    continuity_constr = m.addConstr( gp.quicksum(d_vars[prod] for prod in decision_names[0:-1]) - d_vars['FINPROD'] == 0)
    print(gp.quicksum(d_vars[prod] for prod in decision_names[0:-1]) - d_vars['FINPROD'] == 0)
    veg_production_constr = m.addConstr(d_vars["VEG1"] + d_vars['VEG2'] - 200 <= 0)
    non_veg_production_constr = m.addConstr(d_vars['OIL1'] + d_vars['OIL2'] + d_vars['OIL3'] - 250<=0)

    
    
    
    m.update()
    m.optimize()
    
    findResults = m.X
    for i in range(len(findResults)):
        print(f'optimal soln has {findResults[i]} of {decision_names[i]}')
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))



<gurobi.TempConstr: <gurobi.LinExpr: used[VEG1] + used[VEG2] + used[OIL1] + used[OIL2] + used[OIL3] + -1.0 used[FINPROD]> == 0>
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 5 rows, 6 columns and 23 nonzeros
Model fingerprint: 0xb03a5e80
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+02]
Presolve time: 0.01s
Presolved: 5 rows, 6 columns, 23 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5000000e+32   2.125000e+30   1.500000e+02      0s
       3    1.8000000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  1.800000000e+04
optimal soln has 200.0 of VEG1
optimal soln has 0.0 of VEG2
optimal soln has 0.0 of OIL1
optimal soln has 250.0 of OIL2
optimal soln has 0.0 of OIL3
optimal soln has 450.0 of FINPROD


It looks like we built an easy model for any month, but we should actually solve the problem, setting up a six-month plan.

To do that, we'll have to create month-specific decision variables, tie them in together, and make a really big block model!

To account for time, we decompose our decision variables $X_i$ into three:
1. $ X^{(\alpha) (t)}_i$ : represents the amount ingredient i is bought at month $t$  
2. $ X^{(\beta) (t)}_i$ : represents the amount ingredient i is stored after production on month $t$  
3. $ X^{(\delta) (t)}_i$ : represents the amount ingredient i is used during production of a final product on month $t$  


our objective function will change to:
$$max \sum_{\forall t} (-P^t_{x_1}X^{(\alpha) (t)}_1 - P^t_{x_2} X^{(\alpha) (t)}_2 - P^t_{x_3}X^{(\alpha) (t)}_3 - P^t_{x_4}X^{(\alpha) (t)}_4 - P^t_{x_5}X^{(\alpha) (t)}_5 + 150 Y^{(t)}) - \sum_{\forall t} 5(X^{(\beta) (t)}_1+X^{(\beta) (t)}_2+X^{(\beta) (t)}_3+X^{(\beta) (t)}_4+X^{(\beta) (t)}_5) $$

As this $-\sum_{\forall t} 5(X^{(\beta) (t)}_1+X^{(\beta) (t)}_2+X^{(\beta) (t)}_3+X^{(\beta) (t)}_4+X^{(\beta) (t)}_5$ clause represents the cost of having raw ingredients in storage, impacting the final objective. 

In temporalizing $X_i$, we have to represent the product of $Y^{(t)}$ with an affine constraint for each $t$:
$$Y^{(t)} - ( X^{(\delta) (t)}_1 + X^{(\delta) (t)}_2 + X^{(\delta) (t)}_3 + X^{(\delta) (t)}_4 + X^{(\delta) (t)}_5) = 0$$

We also have to temporalize the hardness constraints, using the production  $ X^{(\delta) (t)}_i$, for each $t$:
$$ 8.8X^{(\delta) (t)}_1 + 6.1 X^{(\delta) (t)}_2 + 2.0 X^{(\delta) (t)}_3 + 4.2 X^{(\delta) (t)}_4 + 5.0X^{(\delta) (t)}_5 - 3 Y^{(t) }\geq 0$$
$$ 8.8X^{(\delta) (t)}_1 + 6.1 X^{(\delta) (t)}_2 + 2.0 X^{(\delta) (t)}_3 + 4.2 X^{(\delta) (t)}_4 + 5.0X^{(\delta) (t)}_5 - 6 Y^{(t) }\geq 0$$

We also know that you cannot store more than 1000 tonnes of each oil per month, creating a bunch of inequality constraint, for each $t$, for each ingredient $i$:
$$X^{(\beta) (t)}_i - 1000 \leq 0$$

There's a conceptually interesting representation between treating the amount of $X_i$ is in storage, $X^{(\beta) (t)}_i$, as a decision variable which is mainly determined by a recursive constraint. We know that for t=0, all $X^{(\beta) (t)}_i = 500$.

For any $t > 0$, $X^{(\beta) (t)}_i$ is defined by the following constraint:
$$X^{(\beta) (t)}_i = X^{(\alpha) (t)}_i + X^{(\beta) (t-1)}_i - X^{(\delta) (t)}_i $$. In english, the amount of $X_i$ stored in month $t$ is the amount $X_i$ bought plus the amount $X_i$ we already had in storage, minus the amount we used up to make some final product. 

To enforce that we have 500 tonnes of each oil in June, we set the following constraint when t = 6:
$$X^{(\beta) (t=6)}_i - 500 = 0 $$

There's also the product constraint, that for each month, we cannot refine more than 200 tons of vegtable oils and more than 250 tons of non-vegetable oils. Since there's no loss of weight in the refining process, and that the cost of refinement can be ignored, this materializes into the constraint for each time $t$:
$$ X^{(\delta) (t)}_1 +  X^{(\delta) (t)}_2 - 200 \leq 0$$ 
and 
$$X^{(\delta) (t)}_3 +  X^{(\delta) (t)}_4 +  X^{(\delta) (t)}_5 - 250 \leq 0$$

Moreover, we have to add the logical constraints that we cannot buy or product negative values of ingredients, so our model makes sense! So for all t, for every ingredient i, 
$$X^{(\delta) (t)}_i \geq 0$$
and
$$X^{(\alpha) (t)}_i \geq 0$$
Our hard equality constraint on the $\beta$ terms along with these constraints algebraically ensure we don't store negative values of ingredients.


In [259]:
#first, make the month decision variables
months = ['January','February','March','April', 'May', 'June'  ]
month_final_product_names = {}
for month in months:
    month_final_product_names[month] = months

#month_decision_var_names #uncomment to check if correct



## Setting HyperParameters:

In [281]:
final_prod_val = 150
storage_cost_per_ton = 5
veg_capacity_limit = 200
non_veg_capacity_limit = 250
maximum_product_allowed_in_storage = 1000
initial_amount_in_storage = 500
desired_end_amount_in_storage = 500
hardness_upper_bound = 6
hardness_lower_bound = 3


# Multi-Period Model

In [284]:
try:
    m = gp.Model()
    #initialize block model by adding time-sensitive variables.
   #add decision variables for the month
    month_final_product_variables = m.addVars(month_final_product_names['April'], vtype=GRB.CONTINUOUS,name='FIN_PROD')
    month_bought_decision_variables = m.addVars(prices.keys(), vtype=GRB.CONTINUOUS,name='bought')
    month_store_decision_variables = m.addVars(prices.keys(),vtype=GRB.CONTINUOUS,name='store')
    month_used_decision_variables = m.addVars(prices.keys(),vtype=GRB.CONTINUOUS,name='used')
    
    #add objective function
    #print(month_final_product_variables) #sanity check
    obj = -1*gp.quicksum(
        prices[(current_month,prod)] * month_bought_decision_variables[(current_month,prod)] 
                                for current_month,prod in prices.keys() ) + final_prod_val* month_final_product_variables.sum() -storage_cost_per_ton*month_store_decision_variables.sum()
    m.setObjective( obj, GRB.MAXIMIZE)  
    
    m.update()
    for month in months:
        #print(month_final_product_variables[month] - gp.quicksum( month_used_decision_variables.select(month)) == 0)
        temporal_continuity_constraints[month] = m.addConstr(
            month_final_product_variables[month] - gp.quicksum(
                month_used_decision_variables.select(month)) == 0,'Blending_'+month)
        hardness_lower_constraints = m.addConstr(
            gp.quicksum( 
                hardness_vals[prod]*month_used_decision_variables[month, prod] for prod in decision_names[0:-1] 
                        )  >= hardness_lower_bound * month_final_product_variables[month] ,'Hardness_LB_'+month)
       # print(  gp.quicksum( 
        #        hardness_vals[prod]*month_used_decision_variables[month, prod] for prod in decision_names[0:-1] 
         #               ) -3 * month_final_product_variables[month] <= 0 )
        hardness_upper_constraints = m.addConstr(
            gp.quicksum( 
                hardness_vals[prod]*month_used_decision_variables[month, prod] for prod in decision_names[0:-1]
                        )  >= hardness_upper_bound* month_final_product_variables[month],'Hardness_UB_'+month )
       # print(month_used_decision_variables.select(month,"VEG1")[0] + month_used_decision_variables.select(month,"VEG2")[0] - 200 <= 0)
        vegetable_product_constraints = m.addConstr(
           month_used_decision_variables.select(month,"VEG1")[0] + month_used_decision_variables.select(month,"VEG2")[0] <= veg_capacity_limit
           ,'Capacity_VEG_'+month)
        #print('\n',month_used_decision_variables.select(month,"OIL1")[0] + month_used_decision_variables.select(month,"OIL2")[0]
        #    + month_used_decision_variables.select(month,"OIL3")[0] - 250 <= 0)
        non_veg_product_constraints = m.addConstr(
            month_used_decision_variables.select(month,"OIL1")[0] + month_used_decision_variables.select(month,"OIL2")[0]
            + month_used_decision_variables.select(month,"OIL3")[0] <= non_veg_capacity_limit
            ,'Capacity_OIL_'+month)
    #constraints that depend on month,product    

    
    for month,product in month_store_decision_variables.keys():
        maximum_storage_constraint[(month,product)] =  m.addConstr(month_store_decision_variables[(month,product)] - maximum_product_allowed_in_storage <= 0 ,product+'_Maximum_storage_constraint_'+month)
        positive_production_constraint[(month,product)] = m.addConstr(month_used_decision_variables[(month,product)] >= 0, 'positivity_constr')
        positive_purchase_constraint[(month,product)] = m.addConstr(month_bought_decision_variables[(month,product)] >= 0,'positivity_constr')

    
    #in june, we have 500 tonnes of each oil in storage. Enforced with constraint.

    for  idx, product_decision_var in enumerate(month_store_decision_variables.select('June')):
        jun_storage_constraints[idx] = m.addConstr(product_decision_var - desired_end_amount_in_storage == 0,product+'_June_storage_constr')
    
    #recursive storage value constraints:
    for idx, month in enumerate(months):
        if month == 'January':
            for product in decision_names[0:-1]: 
                storage_equality_constraints[(month,product)] = m.addConstr(
                initial_amount_in_storage + month_bought_decision_variables[(month,product)] == month_used_decision_variables[(month,product)] 
                    + month_store_decision_variables[(month,product)]
                ,product+'_Jan_storage_constr')
        else:
            for product in decision_names[0:-1]:
                storage_equality_constraints[(month,product)] = m.addConstr( 
                    month_store_decision_variables[(month,product)] == month_bought_decision_variables[(month,product)] +
                    month_store_decision_variables[(months[idx-1],product)] - month_used_decision_variables[(month,product)]
                ,product+'_'+month+'_storage_constr')
    m.update()
    m.optimize()
    
   # for idx, val in enumerate(findResults):
   #     print( str(val) + ' ' + key_list[idx])
   
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

    

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 155 rows, 96 columns and 348 nonzeros
Model fingerprint: 0x0bbfe694
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
Presolve removed 101 rows and 11 columns
Presolve time: 0.01s
Presolved: 54 rows, 85 columns, 200 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8475779e+05   1.284563e+03   0.000000e+00      0s
      54    1.0787963e+05   0.000000e+00   0.000000e+00      0s

Solved in 54 iterations and 0.02 seconds
Optimal objective  1.078796296e+05


In [271]:
for v in m.getVars():
    print("%s %f" % (v.Varname, v.X))
for c in m.getConstrs():
    if c.Slack > 0:
        print(f'{c.ConstrName}, has slack:{c.Slack}')

FIN_PROD[January] 450.000000
FIN_PROD[February] 450.000000
FIN_PROD[March] 450.000000
FIN_PROD[April] 450.000000
FIN_PROD[May] 450.000000
FIN_PROD[June] 450.000000
bought[January,VEG1] 0.000000
bought[January,VEG2] 0.000000
bought[January,OIL1] 0.000000
bought[January,OIL2] 0.000000
bought[January,OIL3] 0.000000
bought[February,VEG1] 0.000000
bought[February,VEG2] 0.000000
bought[February,OIL1] 0.000000
bought[February,OIL2] 0.000000
bought[February,OIL3] 0.000000
bought[March,VEG1] 0.000000
bought[March,VEG2] 0.000000
bought[March,OIL1] 0.000000
bought[March,OIL2] 0.000000
bought[March,OIL3] 750.000000
bought[April,VEG1] 0.000000
bought[April,VEG2] 0.000000
bought[April,OIL1] 0.000000
bought[April,OIL2] 0.000000
bought[April,OIL3] 0.000000
bought[May,VEG1] 74.074074
bought[May,VEG2] 0.000000
bought[May,OIL1] 0.000000
bought[May,OIL2] 0.000000
bought[May,OIL3] 0.000000
bought[June,VEG1] 700.000000
bought[June,VEG2] 425.925926
bought[June,OIL1] 0.000000
bought[June,OIL2] 750.000000
boug

[(0, <gurobi.Var store[January,VEG1] (value 500.0)>), (1, <gurobi.Var store[January,VEG2] (value 419.3548387096774)>), (2, <gurobi.Var store[January,OIL1] (value 250.0)>), (3, <gurobi.Var store[January,OIL2] (value 500.0)>), (4, <gurobi.Var store[January,OIL3] (value 500.0)>)]


<gurobi.LinExpr: -110.0 bought[January,VEG1] + -120.0 bought[January,VEG2] + -130.0 bought[January,OIL1] + -110.0 bought[January,OIL2] + -115.0 bought[January,OIL3] + -130.0 bought[February,VEG1] + -130.0 bought[February,VEG2] + -110.0 bought[February,OIL1] + -90.0 bought[February,OIL2] + -115.0 bought[February,OIL3] + -110.0 bought[March,VEG1] + -140.0 bought[March,VEG2] + -130.0 bought[March,OIL1] + -100.0 bought[March,OIL2] + -95.0 bought[March,OIL3] + -120.0 bought[April,VEG1] + -110.0 bought[April,VEG2] + -120.0 bought[April,OIL1] + -120.0 bought[April,OIL2] + -125.0 bought[April,OIL3] + -100.0 bought[May,VEG1] + -120.0 bought[May,VEG2] + -150.0 bought[May,OIL1] + -110.0 bought[May,OIL2] + -105.0 bought[May,OIL3] + -90.0 bought[June,VEG1] + -100.0 bought[June,VEG2] + -140.0 bought[June,OIL1] + -80.0 bought[June,OIL2] + -135.0 bought[June,OIL3] + 150.0 FIN_PROD[January] + 150.0 FIN_PROD[February] + 150.0 FIN_PROD[March] + 150.0 FIN_PROD[April] + 150.0 FIN_PROD[May] + 150.0 FIN_PROD